In [1]:
import pymongo
import pandas as pd
import matplotlib as plt
from pandas import Series,DataFrame
pd.set_option('display.width', None)  # 设置字符显示宽度
pd.set_option('display.max_rows', None)  # 设置显示最大行
pd.set_option('display.max_columns', None)  # 设置显示最大行
#二行即可搞定画图中文乱码
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

date = '20190812'
#连接数据库
client = pymongo.MongoClient('localhost',27017)
futures = client.futures
market = futures.indexMarket
unit = futures.unit
position = futures.position

#加载数据
market = DataFrame(list(market.find({'date': {'$gte': date}})))
unit = DataFrame(list(unit.find()))
position = DataFrame(list(position.find({'date': {'$gte': date}})))

#大写字母
position=position.rename(columns={'var':'variety'})

#删除/选取某行含有特殊数值的列
position=position.dropna()
#选择需要显示的字段

data1=market[['date','variety','set_close']]

data2=unit[[ 'variety','unit']]
position=position[['date','variety','symbol','long_party_name','long_openIntr', 'long_openIntr_chg','short_party_name','short_openIntr', 'short_openIntr_chg']]

#查询会员
members='海通期货'
data3=position[(position['long_party_name'] == members)]

#汇总合约
data3=data3[['date','variety','long_party_name','long_openIntr']]

data3=data3.groupby(['date','variety','long_party_name'])[['long_openIntr']].sum()

data4=position[(position['short_party_name'] == members)]
data4=data4[['date','variety','short_party_name','short_openIntr']]
data4=data4.groupby(['date','variety','short_party_name'])[['short_openIntr']].sum()


#并集
data5=pd.merge(data3,data4, on=['date','variety'],how='outer')
data5['会员简称']=data5.apply(lambda x: members,axis=1)

#nan缺失值填充fillna()为0
data5=data5.fillna(0)
data5['净持仓']=data5.apply(lambda x: x['long_openIntr']-x['short_openIntr'],axis=1)

# df['当日涨幅']=df.apply(lambda x: x['收盘']-x['收盘'].shift(1)-1,axis=1)
#选择需要显示的字段
data5=data5[['会员简称','long_openIntr','short_openIntr','净持仓']]
data5=data5.reset_index(['variety','date'])
netpostion=data5.set_index('date')

netpostion=data5.pivot_table('净持仓',index='date',columns='variety',fill_value=0)

#合约价值
contractValue=pd.merge(data1,data2,how='left',sort=False).drop_duplicates()
contractValue['contractValue'] = contractValue.apply(lambda x: x['set_close']*x['unit'],axis=1)
contractValue=contractValue[['date','variety','contractValue']]

#值替换replace()
# contractValue=contractValue.replace(['TA'],'PTA')
sz=pd.merge(data5,contractValue,on=['date','variety'],how='left')

#净持仓价值
sz['净持仓价值']=sz.apply(lambda x: x['净持仓']*x['contractValue'],axis=1)
sz=sz[['date','variety','会员简称','净持仓价值']]
sz=sz.sort_values(by='净持仓价值')
sz=sz.pivot_table('净持仓价值',index=['date','会员简称'],columns='variety',fill_value=0)
sz=sz.copy()
#所有品种
sz
sz['化工板块']=sz[['RU','MA','V','L','PP','BU','TA']].sum(axis=1)
sz['油脂板块']=sz[['Y','P','OI']].sum(axis=1)
sz['有色板块']=sz[['CU','AL','ZN','NI','PB']].sum(axis=1)
sz['黑色板块']=sz[['RB','HC','ZC','J','JM','I']].sum(axis=1)
sz['黄金白银']=sz[['AU','AG']].sum(axis=1)
sz['工业品']=sz[['CU','AL','ZN','NI','PB','RB','FG','RU','L','V','TA','MA','PP','JM','J','ZC','I','BU','HC','SM','FU']].sum(axis=1)
sz['农产品']=sz[['A','C','M','RM','Y','P','OI','CF','SR','JD','CS','AP']].sum(axis=1)
sz['商品板块']=sz.sum(axis=1)


chemical=sz.copy()
chemical=chemical[['RU','MA','V','L','PP','BU','TA','商品板块','工业品','化工板块']]
chemical['max']=chemical[['RU','MA','V','L','PP','BU','TA']].idxmax(axis=1)
chemical['min']=chemical[['RU','MA','V','L','PP','BU']].idxmin(axis=1)
chemical['20190920':]



,variety,RU,MA,V,L,PP,BU,TA,商品板块,工业品,化工板块,max,min
date,会员简称,,,,,,,,,,,,
20190920,海通期货,-799167600,22839180,0,498086400,225910000,-127382530,-551954295,-3367909283,-2090441051,-731668845,L,RU
20190923,海通期货,-857010960,165942600,0,654374000,0,0,508981530,902773365,-243088285,472287170,L,RU
20190924,海通期货,-967891680,163540080,0,391617030,0,148707000,-65240430,-3829352983,-2584954486,-329268000,L,RU
20190925,海通期货,-1042963470,159498430,0,309569320,330986565,303896740,501787305,-1780657511,-1296392817,562774890,TA,RU
20190926,海通期货,-1042629200,-93844220,0,36813700,332459820,303475130,88886850,-14986909680,-3678402103,-374837920,PP,RU
20190927,海通期货,-985068810,-186478880,0,47301150,220268640,125696610,616100210,-11863777349,-2225288250,-162181080,TA,RU
20190930,海通期货,-880631850,86940540,0,29221680,0,130791060,417302400,-13871758457,-2878440759,-216376170,TA,RU
20191008,海通期货,-996712620,72083340,0,0,300493090,192967240,561371250,-12392693139,-2085598610,130202300,TA,RU
20191009,海通期货,-917386380,164659320,0,0,253025460,136452330,723469915,-13120403777,-3427403563,360220645,TA,RU


In [2]:
import json
from pymongo import MongoClient
# 连接数据库
client = MongoClient('localhost', 27017)
db = client.futures
flow = db.flows
chemical = chemical.reset_index()

futures = client.futures
flows = futures.flows
flows.insert_many(json.loads(chemical.T.to_json()).values())
print("写入OK")

写入OK


In [2]:
黑色板块=sz.copy()
黑色板块=黑色板块[['RB','HC','ZC','J','JM','I','黑色板块','工业品','商品板块']] 
黑色板块['max']=黑色板块[['RB','HC','ZC','J','JM','I']].idxmax(axis=1) 
黑色板块['min']=黑色板块[['RB','HC','ZC','J','JM','I']].idxmin(axis=1)
黑色板块['20190920':]

,variety,RB,HC,ZC,J,JM,I,黑色板块,工业品,商品板块,max,min
date,会员简称,,,,,,,,,,,
20190920,海通期货,-567214800,-115428240,0,62952500,-93639060,-501732000,-1215061600,-2090441051,-3367909283,J,RB
20190923,海通期货,-552930780,-125554000,198316800,318582000,-12830400,-302827500,-477243880,-243088285,902773365,J,RB
20190924,海通期货,-491929380,-150588900,182677500,-638826500,-183912960,-440418500,-1722998740,-2584954486,-3829352983,ZC,J
20190925,海通期货,-303874300,-139744350,43985800,24053800,-245628600,-92032200,-713239850,-1296392817,-1780657511,ZC,RB
20190926,海通期货,-632270520,-59305180,-178020000,-449999600,-354791760,-344198600,-2018585660,-3678402103,-14986909680,HC,RB
20190927,海通期货,-544434960,-38436840,-188339000,-257487600,-253825380,-205376000,-1487899780,-2225288250,-11863777349,HC,RB
20190930,海通期货,-904651600,96111360,-186659900,-407316700,-395924760,-400811400,-2199253000,-2878440759,-13871758457,HC,RB
20191008,海通期货,-985070720,80143180,-212602400,-212214000,-337471080,-489065200,-2156280220,-2085598610,-12392693139,HC,RB
20191009,海通期货,-976446000,-38441250,-194764500,-970170300,-280125720,-844486500,-3304434270,-3427403563,-13120403777,HC,RB


In [4]:
import json
from pymongo import MongoClient
# 连接数据库
client = MongoClient('localhost', 27017)
db = client.futures
flow = db.flows
黑色板块=sz.copy()
黑色板块=黑色板块[['RB','HC','ZC','J','JM','I','黑色板块','工业品','商品板块']] 
黑色板块['max']=黑色板块[['RB','HC','ZC','J','JM','I']].idxmax(axis=1) 
黑色板块['min']=黑色板块[['RB','HC','ZC','J','JM','I']].idxmin(axis=1)
df = 黑色板块.reset_index()
flow.insert_many(json.loads(df.T.to_json()).values())
print("写入OK")

写入OK


In [3]:
oil=sz.copy()
oil=oil[['Y','P','OI','油脂板块','农产品']]
oil['max']=oil[['Y','P','OI']].idxmax(axis=1) 
oil['min']=oil[['Y','P','OI']].idxmin(axis=1)
oil['20190920':]

,variety,Y,P,OI,油脂板块,农产品,max,min
date,会员简称,,,,,,,
20190920,海通期货,236490480,248209840,-167490600,317209720,268123215,P,OI
20190923,海通期货,205590900,288043350,0,493634250,-461551900,P,OI
20190924,海通期货,47920000,231041130,-180223850,98737280,41765095,P,OI
20190925,海通期货,102623950,0,-174435840,-71811890,1042793570,Y,OI
20190926,海通期货,25171960,0,-162065280,-136893320,-1319448910,Y,OI
20190927,海通期货,-523773710,156995640,-145625350,-512403420,-1649697890,P,Y
20190930,海通期货,-484581230,151850900,-132267850,-464998180,-1550939645,P,Y
20191008,海通期货,-607027140,154900270,-176945760,-629072630,-1423140775,P,Y
20191009,海通期货,-642128270,206059040,-158108240,-594177470,-208484145,P,Y


In [4]:
有色=sz.copy()
有色=有色[['CU','AL','ZN','NI','PB','有色板块','工业品','商品板块']]
有色['max']=有色[['CU','AL','ZN','NI','PB']].idxmax(axis=1) 
有色['min']=有色[['CU','AL','ZN','NI','PB']].idxmin(axis=1)
有色['20190920':]

,variety,CU,AL,ZN,NI,PB,有色板块,工业品,商品板块,max,min
date,会员简称,,,,,,,,,,
20190920,海通期货,-391716780,10813280,15781500,303111704,160849920,98839624,-2090441051,-3367909283,NI,CU
20190923,海通期货,-441713220,69592215,-55981530,190112160,210323080,-27667295,-243088285,902773365,PB,CU
20190924,海通期货,-257088695,85384800,-171467875,67044164,229808540,-46319066,-2584954486,-3829352983,PB,CU
20190925,海通期货,-640858635,119000980,-130391460,-314462392,269232930,-697478577,-1296392817,-1780657511,PB,CU
20190926,海通期货,-543247480,96735600,-167724075,-771858788,272986200,-1113108543,-3678402103,-14986909680,PB,NI
20190927,海通期货,-866074695,116915000,3653715,219357990,282668660,-243479330,-2225288250,-11863777349,PB,CU
20190930,海通期货,-698827500,27735840,-141740895,226612176,415641600,-170578779,-2878440759,-13871758457,PB,CU
20191008,海通期货,-384654900,28353010,-180406750,326570895,418652230,208514485,-2085598610,-12392693139,PB,CU
20191009,海通期货,-701826650,16706400,-227696480,364832992,424671870,-123311868,-3427403563,-13120403777,PB,CU


In [5]:
农产品=sz.copy()
农产品=农产品[['A','C','M','RM','Y','P','OI','CF','SR','JD','CS','AP','农产品']]
农产品['max']=农产品[['A','C','M','RM','Y','P','OI','CF','SR','JD','CS','AP']].idxmax(axis=1) 
农产品['min']=农产品[['A','C','M','RM','Y','P','OI','CF','SR','JD','CS','AP']].idxmin(axis=1)
农产品['20190920':]

,variety,A,C,M,RM,Y,P,OI,CF,SR,JD,CS,AP,农产品,max,min
date,会员简称,,,,,,,,,,,,,,,
20190920,海通期货,111016320,-375515400,358513250,0,236490480,248209840,-167490600,-519732675,552991320,110193760,-50734500,-235818580,268123215,SR,CF
20190923,海通期货,120200800,-382436180,119658000,25502750,205590900,288043350,0,-677341980,0,108248120,-53424000,-215593660,-461551900,P,CF
20190924,海通期货,30263970,-312366960,244772580,139541560,47920000,231041130,-180223850,-688653315,596871800,121108320,-85050740,-103459400,41765095,SR,CF
20190925,海通期货,38834200,-266465920,1051736020,191436960,102623950,0,-174435840,-604745400,733512300,106589200,-93800200,-42491700,1042793570,M,CF
20190926,海通期货,18057580,-248681300,-447420800,-133417200,25171960,0,-162065280,-640491620,209059720,234698400,-68429010,-105931360,-1319448910,JD,CF
20190927,海通期货,-15130000,-249823700,-556281400,-111487200,-523773710,156995640,-145625350,-490085900,261324540,249855270,-73347260,-152318820,-1649697890,SR,M
20190930,海通期货,-23532700,-225974880,0,86449400,-484581230,151850900,-132267850,-892946505,-22825000,168860540,-58397040,-117575280,-1550939645,JD,CF
20191008,海通期货,-36320400,-247768470,202964700,41253300,-607027140,154900270,-176945760,-997009575,46439800,182916160,-53270320,66726660,-1423140775,M,CF
20191009,海通期货,-36193500,-269214400,433267380,150355600,-642128270,206059040,-158108240,-296862345,289595340,136506040,-45839390,24078600,-208484145,M,Y


In [6]:
板块=sz.copy()
板块=板块[['商品板块','工业品','化工板块','黑色板块','有色板块','农产品','油脂板块']]
板块['20190920':]

,variety,商品板块,工业品,化工板块,黑色板块,有色板块,农产品,油脂板块
date,会员简称,,,,,,,
20190920,海通期货,-3367909283,-2090441051,-731668845,-1215061600,98839624,268123215,317209720
20190923,海通期货,902773365,-243088285,472287170,-477243880,-27667295,-461551900,493634250
20190924,海通期货,-3829352983,-2584954486,-329268000,-1722998740,-46319066,41765095,98737280
20190925,海通期货,-1780657511,-1296392817,562774890,-713239850,-697478577,1042793570,-71811890
20190926,海通期货,-14986909680,-3678402103,-374837920,-2018585660,-1113108543,-1319448910,-136893320
20190927,海通期货,-11863777349,-2225288250,-162181080,-1487899780,-243479330,-1649697890,-512403420
20190930,海通期货,-13871758457,-2878440759,-216376170,-2199253000,-170578779,-1550939645,-464998180
20191008,海通期货,-12392693139,-2085598610,130202300,-2156280220,208514485,-1423140775,-629072630
20191009,海通期货,-13120403777,-3427403563,360220645,-3304434270,-123311868,-208484145,-594177470
